### Preparation

In [1]:
!pip3 install transformers datasets pandas evaluate numpy

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import numpy as np
import evaluate

In [2]:
df = pd.read_csv("/kaggle/input/hacks-2/train_dataset_train.csv", sep=';')
df = df[['Текст инцидента', 'Группа тем']]
df.columns = ['text', 'label']
df.head()

,text,label
0,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",Благоустройство
1,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Социальное обслуживание и защита
2,'Добрый день ! Скажите пожалуйста если подовал...,Социальное обслуживание и защита
3,'Каждая из них не о чем. Люди на остановках хо...,Общественный транспорт
4,'В Березниках у сына привитого откоронавируса ...,Здравоохранение/Медицина


In [3]:
labels = list(df.label.unique())
label2id = dict(zip(labels, list(range(len(labels)))))
id2label = dict(zip(list(range(len(labels))), labels))
n_labels = len(label2id)

In [5]:
df['label'] = df.apply(lambda row: label2id[row['label']], axis=1)
df

,text,label
0,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",0
1,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",1
2,'Добрый день ! Скажите пожалуйста если подовал...,1
3,'Каждая из них не о чем. Люди на остановках хо...,2
4,'В Березниках у сына привитого откоронавируса ...,3
...,...,...
23123,'Город Гремячинск-— ситуация с теплом на улице...,7
23124,"'Здравствуйте у меня ребёнку 2 месяца , тест н...",3
23125,'А что творится с благоустройством дворов! Воо...,0
23126,'Сообщение без текста,7


### Training model for large labels

In [33]:
model = AutoModelForSequenceClassification.from_pretrained(
    "cointegrated/rubert-tiny2", num_labels=len(id2label.keys()), id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
def token(text):
    return tokenizer(text['text'], padding=True, truncation=True, max_length=512)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], random_state=42,
                                                   shuffle=True, stratify=df['label'], test_size=0.2)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [37]:
df_train = pd.DataFrame(data={
    'text': X_train,
    'label': y_train
})
df_test = pd.DataFrame(data={
    'text': X_test,
    'label': y_test
})

In [38]:
train = Dataset.from_pandas(df_train)
test = Dataset.from_pandas(df_test)
tokenized_train = train.map(token)
tokenized_test = test.map(token)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/18502 [00:00<?, ?ex/s]

  0%|          | 0/4626 [00:00<?, ?ex/s]

In [41]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_args = TrainingArguments(
    output_dir="/kaggle/working/",
    learning_rate=2e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    optim = 'adafactor',
    lr_scheduler_type = 'cosine',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [42]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,1.079500,0.822761,0.739423
2,0.636800,0.782963,0.765267
3,0.432900,0.864569,0.763748
4,0.273400,1.097742,0.750224
5,0.170400,1.214440,0.754050
6,0.114600,1.300219,0.754886
7,0.060000,1.353627,0.757392
8,0.050900,1.378456,0.758785
9,0.051000,1.387636,0.757322


KeyboardInterrupt: 

In [51]:
df_aug = pd.read_csv("/kaggle/input/hacks-2-augmented/augmented (1).csv")
df_aug = df_aug[['text', 'big_labels']]
df_aug.columns = ['text', 'label']

df_aug['label'] = df_aug.apply(lambda row: label2id[row['label']], axis=1)

df_aug

,text,label
0,"В яму упала мать, упала ребенок. Сильно ударил...",0
1,В Перми с ноября года не работает социальное т...,1
2,"Если подавала на пособие с до декабря, когда м...",1
3,На остановках можно будет только укрыться от н...,2
4,В Березниках у сына принятого откоронавируса з...,3
...,...,...
23589,"город Пермь. Добрый вечер, адрес: ул. Водонагр...",14
23590,Мир вам. Что значит побег? Почему вам следует ...,14
23591,город Пермь. Спокойной ночи. Шум в этом году н...,14
23592,город Пермь. Куда писать негативные отзывы о б...,14


In [52]:
train_aug = Dataset.from_pandas(df_aug)
tokenized_train_aug = train_aug.map(token)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/23594 [00:00<?, ?ex/s]

In [53]:
training_args_aug = TrainingArguments(
    output_dir="/kaggle/working/",
    learning_rate=2e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    optim = 'adafactor',
    lr_scheduler_type = 'cosine',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
trainer_aug = Trainer(
    model=model,
    args=training_args_aug,
    train_dataset=tokenized_train_aug,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [54]:
trainer_aug.train()

Epoch,Training Loss,Validation Loss,F1
1,0.749600,0.699047,0.787812
2,0.480700,0.588044,0.835945
3,0.140000,0.599849,0.850948
4,0.091700,0.642246,0.853034
5,0.033100,0.645316,0.855940


TrainOutput(global_step=3690, training_loss=0.25787200346225647, metrics={'train_runtime': 217.464, 'train_samples_per_second': 542.481, 'train_steps_per_second': 16.968, 'total_flos': 212003900994480.0, 'train_loss': 0.25787200346225647, 'epoch': 5.0})

### Training model for small labels

In [77]:
df_small = pd.read_csv("/kaggle/input/hacks-2/train_dataset_train.csv", sep=';')
df_small['Текст инцидента'] = df_small['Группа тем'] + ": " + df_small['Текст инцидента']
df_small = df_small[['Текст инцидента', 'Тема']]
df_small.columns = ['text', 'label']
df_small.head()

,text,label
0,"Благоустройство: 'Добрый день. Сегодня, 20.08....",★ Ямы во дворах
1,"Социальное обслуживание и защита: 'Пермь г, +7...",Оказание гос. соц. помощи
2,Социальное обслуживание и защита: 'Добрый день...,Дети и многодетные семьи
3,Общественный транспорт: 'Каждая из них не о че...,Содержание остановок
4,Здравоохранение/Медицина: 'В Березниках у сына...,Технические проблемы с записью на прием к врачу


In [78]:
labels_small = list(df_small.label.unique())
label2id_small = dict(zip(labels_small, list(range(len(labels_small)))))
id2label_small = dict(zip(list(range(len(labels_small))), labels_small))
n_labels_small = len(label2id_small)

In [79]:
df_small['label'] = df_small.apply(lambda row: label2id_small[row['label']], axis=1)
df_small

,text,label
0,"Благоустройство: 'Добрый день. Сегодня, 20.08....",0
1,"Социальное обслуживание и защита: 'Пермь г, +7...",1
2,Социальное обслуживание и защита: 'Добрый день...,2
3,Общественный транспорт: 'Каждая из них не о че...,3
4,Здравоохранение/Медицина: 'В Березниках у сына...,4
...,...,...
23123,ЖКХ: 'Город Гремячинск-— ситуация с теплом на ...,44
23124,Здравоохранение/Медицина: 'Здравствуйте у меня...,4
23125,Благоустройство: 'А что творится с благоустрой...,122
23126,ЖКХ: 'Сообщение без текста,44


In [80]:
model_small = AutoModelForSequenceClassification.from_pretrained(
    "cointegrated/rubert-tiny2", num_labels=len(id2label_small.keys()), id2label=id2label_small, label2id=label2id_small
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(df_small['text'], df_small['label'], random_state=42,
                                                   shuffle=True, stratify=df_small['label'], test_size=0.2)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [82]:
df_train_small = pd.DataFrame(data={
    'text': X_train_small,
    'label': y_train_small
})
df_test_small = pd.DataFrame(data={
    'text': X_test_small,
    'label': y_test_small
})

In [83]:
train_small = Dataset.from_pandas(df_train_small)
test_small = Dataset.from_pandas(df_test_small)
tokenized_train_small = train_small.map(token)
tokenized_test_small = test_small.map(token)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/18502 [00:00<?, ?ex/s]

  0%|          | 0/4626 [00:00<?, ?ex/s]

In [84]:
data_collator_small = DataCollatorWithPadding(tokenizer=tokenizer)
training_args_small = TrainingArguments(
    output_dir="/kaggle/working/",
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=15,
    weight_decay=0.01,
    optim = 'adafactor',
    lr_scheduler_type = 'cosine',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
trainer_small = Trainer(
    model=model_small,
    args=training_args_small,
    train_dataset=tokenized_train_small,
    eval_dataset=tokenized_test_small,
    tokenizer=tokenizer,
    data_collator=data_collator_small,
    compute_metrics=compute_metrics,
)

In [85]:
trainer_small.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,3.831187,0.126346
2,4.119500,3.108922,0.171430
3,4.119500,2.766903,0.175834
4,2.948800,2.568954,0.191269
5,2.948800,2.425164,0.256639
6,2.555000,2.313804,0.275760
7,2.328000,2.229666,0.302755
8,2.328000,2.167784,0.315111
9,2.188900,2.123389,0.329787
10,2.188900,2.091988,0.342832


TrainOutput(global_step=4350, training_loss=2.4981100990032328, metrics={'train_runtime': 945.0729, 'train_samples_per_second': 293.66, 'train_steps_per_second': 4.603, 'total_flos': 1299839113668708.0, 'train_loss': 2.4981100990032328, 'epoch': 15.0})

In [86]:
df_aug_small = pd.read_csv("/kaggle/input/hacks-2-augmented/augmented (1).csv")
df_aug_small['text'] = df_aug_small['big_labels'] + ": " + df_aug_small['text']
df_aug_small = df_aug_small[['text', 'smol_labels']]
df_aug_small.columns = ['text', 'label']

df_aug_small['label'] = df_aug_small.apply(lambda row: label2id_small[row['label']], axis=1)
df_aug_small

,text,label
0,"Благоустройство: В яму упала мать, упала ребен...",0
1,Социальное обслуживание и защита: В Перми с но...,1
2,Социальное обслуживание и защита: Если подавал...,2
3,Общественный транспорт: На остановках можно бу...,3
4,Здравоохранение/Медицина: В Березниках у сына ...,4
...,...,...
23589,"Спецпроекты: город Пермь. Добрый вечер, адрес:...",81
23590,Спецпроекты: Мир вам. Что значит побег? Почему...,81
23591,Спецпроекты: город Пермь. Спокойной ночи. Шум ...,81
23592,Спецпроекты: город Пермь. Куда писать негативн...,81


In [87]:
train_aug_small = Dataset.from_pandas(df_aug_small)
tokenized_train_aug_small = train_aug_small.map(token)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/23594 [00:00<?, ?ex/s]

In [88]:
training_args_aug_small = TrainingArguments(
    output_dir="/kaggle/working/",
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=10,
    weight_decay=0.01,
    optim = 'adafactor',
    lr_scheduler_type = 'cosine',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
trainer_aug_small = Trainer(
    model=model_small,
    args=training_args_aug_small,
    train_dataset=tokenized_train_aug_small,
    eval_dataset=tokenized_test_small,
    tokenizer=tokenizer,
    data_collator=data_collator_small,
    compute_metrics=compute_metrics,
)

In [89]:
trainer_aug_small.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,1.956688,0.369378
2,2.071800,1.871166,0.396437
3,1.912100,1.792891,0.424759
4,1.912100,1.746239,0.428695
5,1.791600,1.701304,0.446770
6,1.714100,1.666448,0.462046
7,1.643600,1.647607,0.466851
8,1.643600,1.639093,0.465916
9,1.615000,1.633663,0.468559
10,1.601600,1.633718,0.467851


TrainOutput(global_step=3690, training_loss=1.7554215842146215, metrics={'train_runtime': 427.9089, 'train_samples_per_second': 551.379, 'train_steps_per_second': 8.623, 'total_flos': 540187986332016.0, 'train_loss': 1.7554215842146215, 'epoch': 10.0})